In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install faiss-gpu
!pip install faiss-cpu
!pip install transformers datasets
!pip install datasets
!pip install transformers torch

In [2]:
import pandas as pd

# Load dataset from CSV
file_path = '/content/drive/MyDrive/train_expanded.csv'
df = pd.read_csv(file_path)

# Prepare dataset in the expected format
docs = [{"title": q, "text": a} for q, a in zip(df['question'], df['answer'])]


In [3]:
from datasets import Dataset

dataset = Dataset.from_pandas(pd.DataFrame(docs))


In [6]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')
embeddings = model.encode(dataset['text'], show_progress_bar=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.6 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
dataset = dataset.add_column('embeddings', embeddings.tolist())


In [8]:
dataset_path = "/content/drive/MyDrive/rag_dataset"
index_path = "/content/drive/MyDrive/rag_index"

dataset.save_to_disk(dataset_path)
dataset.add_faiss_index(column="embeddings")
dataset.get_index('embeddings').save(index_path)


Saving the dataset (0/1 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration

# Initialize tokenizer and RAG model
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
rag_model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq")

# Initialize retriever with the saved dataset and index paths
retriever = RagRetriever.from_pretrained(
    "facebook/rag-token-nq",
    index_name="custom",
    passages_path=dataset_path,
    index_path=index_path,
    use_dummy_dataset=False
)

# Set the retriever to the rag_model
rag_model.set_retriever(retriever)


config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:175: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


(…)_encoder_tokenizer/tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

question_encoder_tokenizer/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)ncoder_tokenizer/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

generator_tokenizer/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)erator_tokenizer/special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/rag-token-nq were not used when initializing RagTokenForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagTokenForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagTokenForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called fr

In [24]:
#  Function to handle customer queries
def handle_customer_query(query):
    inputs = tokenizer(query, return_tensors="pt")
    outputs = rag_model.generate(input_ids=inputs['input_ids'],
                                 attention_mask=inputs['attention_mask'],
                                 max_length=150,
                                 num_return_sequences=1,
                                 early_stopping=True)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return response

# Example interaction loop
if __name__ == "__main__":
    print("Welcome to the Fruit Shop Chatbot!")
    while True:
        query = input("Customer: ")
        if query.lower() in ["exit", "quit", "bye"]:
            print("Chatbot: Goodbye!")
            break
        response = handle_customer_query(query)
        print(f"Chatbot: {response}")

Welcome to the Fruit Shop Chatbot!
Customer: What should I do if my package is lost or damaged?
Chatbot:  can i return the product
Customer: exit
Chatbot: Goodbye!


**langchain **